# Otimização de Programação de Férias usando MILP

## Objetivo

**Input:** Número de dias corridos de férias desejados (ex: 10 dias)

**Output:** Encontrar a janela de 10 dias corridos que **maximiza** o período total de folga, aproveitando fins de semana e feriados adjacentes.

## Exemplo Correto

- Input: 10 dias corridos de férias
- Janela selecionada: 18/fev a 27/fev (10 dias corridos)
- **Aproveita antes:** 14-15/fev (fim de semana) + 16-17/fev (Carnaval)
- **Período total de folga:** 14/fev a 27/fev = **14 dias totais!**
- Dias úteis usados: apenas os que estão dentro da janela

## 1. Importações e Configurações

In [ ]:
from datetime import datetime, timedelta
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpStatus
import pandas as pd

# Ano fixo
ANO = 2026

## 2. Definição de Feriados Nacionais (2026)

In [ ]:
# Feriados nacionais fixos e móveis para 2026 (Brasil)
FERIADOS_FIXOS = [
    datetime(2026, 1, 1),   # Ano Novo
    datetime(2026, 4, 21),  # Tiradentes
    datetime(2026, 5, 1),   # Dia do Trabalhador
    datetime(2026, 9, 7),   # Independência
    datetime(2026, 10, 12), # Nossa Senhora Aparecida
    datetime(2026, 11, 2),  # Finados
    datetime(2026, 11, 15), # Proclamação da República
    datetime(2026, 12, 25), # Natal
]

# Feriados móveis
FERIADOS_MOVEIS = [
    datetime(2026, 2, 16),  # Carnaval (segunda)
    datetime(2026, 2, 17),  # Carnaval (terça)
    datetime(2026, 4, 3),   # Sexta-feira Santa
]

FERIADOS = FERIADOS_FIXOS + FERIADOS_MOVEIS

print(f"Total de feriados em {ANO}: {len(FERIADOS)}")
for feriado in sorted(FERIADOS):
    print(f"  {feriado.strftime('%d/%m/%Y')}")

## 3. Geração do Calendário

In [ ]:
def gerar_calendario(ano):
    inicio_ano = datetime(ano, 1, 1)
    fim_ano = datetime(ano, 12, 31)
    
    calendario = []
    classificacao = {}
    
    data_atual = inicio_ano
    while data_atual <= fim_ano:
        calendario.append(data_atual)
        dia_semana = data_atual.weekday()
        
        if data_atual.date() in [f.date() for f in FERIADOS]:
            classificacao[data_atual] = 'feriado'
        elif dia_semana >= 5:
            classificacao[data_atual] = 'fim_semana'
        else:
            classificacao[data_atual] = 'dia_util'
        
        data_atual += timedelta(days=1)
    
    return calendario, classificacao

calendario, classificacao = gerar_calendario(ANO)
print(f"Calendário {ANO} gerado: {len(calendario)} dias")

## 4. Modelagem MILP - Maximizar Período Total de Folga

In [ ]:
def calcular_periodo_total_folga(inicio_idx, dias_corridos, calendario, classificacao):
    """
    Calcula o período TOTAL de folga incluindo fins de semana e feriados adjacentes.
    
    Exemplo:
    - Janela: 18/fev a 27/fev (10 dias corridos)
    - Expande para trás: 14-15/fev (fim de semana) + 16-17/fev (Carnaval)
    - Período total: 14/fev a 27/fev = 14 dias!
    """
    fim_idx = inicio_idx + dias_corridos - 1
    
    # Expandir para trás
    idx_inicio_expandido = inicio_idx
    while idx_inicio_expandido > 0:
        data_anterior = calendario[idx_inicio_expandido - 1]
        if classificacao.get(data_anterior) in ['fim_semana', 'feriado']:
            idx_inicio_expandido -= 1
        else:
            break
    
    # Expandir para frente
    idx_fim_expandido = fim_idx
    while idx_fim_expandido < len(calendario) - 1:
        data_proxima = calendario[idx_fim_expandido + 1]
        if classificacao.get(data_proxima) in ['fim_semana', 'feriado']:
            idx_fim_expandido += 1
        else:
            break
    
    return idx_fim_expandido - idx_inicio_expandido + 1


def resolver_otimizacao_férias(dias_corridos_solicitados, calendario, classificacao):
    """
    Resolve usando MILP maximizando o período total de folga.
    """
    n_dias_ano = len(calendario)
    
    # Pré-calcular para cada janela possível:
    # 1. Dias úteis consumidos
    # 2. Período TOTAL de folga (com expansão)
    janela_possivel = {}
    for i in range(n_dias_ano - dias_corridos_solicitados + 1):
        dias_janela = calendario[i:i + dias_corridos_solicitados]
        dias_uteis_janela = sum(1 for d in dias_janela if classificacao[d] == 'dia_util')
        periodo_total = calcular_periodo_total_folga(i, dias_corridos_solicitados, calendario, classificacao)
        
        janela_possivel[i] = {
            'dias_uteis': dias_uteis_janela,
            'periodo_total': periodo_total
        }
    
    # Criar modelo MAXIMIZANDO período total
    modelo = LpProblem("Otimizacao_Férias", LpMaximize)
    y = {i: LpVariable(f"y_{i}", cat='Binary') for i in janela_possivel.keys()}
    
    # Restrição: apenas uma janela
    modelo += lpSum([y[i] for i in janela_possivel.keys()]) == 1
    
    # Objetivo: MAXIMIZAR período total de folga
    modelo += lpSum([janela_possivel[i]['periodo_total'] * y[i] for i in janela_possivel.keys()])
    
    modelo.solve()
    
    return modelo, y, janela_possivel, dias_corridos_solicitados

print("Funções de otimização definidas.")

## 5. Pós-processamento

In [ ]:
def processar_solucao(modelo, y, janela_possivel, dias_corridos, calendario, classificacao):
    if LpStatus[modelo.status] != 'Optimal':
        return None, f"Status: {LpStatus[modelo.status]}"
    
    # Identificar janela selecionada
    inicio_idx = None
    for i in y.keys():
        if y[i].varValue == 1:
            inicio_idx = i
            break
    
    if inicio_idx is None:
        return None, "Nenhuma janela selecionada"
    
    # Janela base (N dias corridos)
    fim_idx = inicio_idx + dias_corridos - 1
    janela_base = calendario[inicio_idx:fim_idx + 1]
    
    # Expandir para calcular período total
    idx_inicio_expandido = inicio_idx
    while idx_inicio_expandido > 0:
        data_anterior = calendario[idx_inicio_expandido - 1]
        if classificacao.get(data_anterior) in ['fim_semana', 'feriado']:
            idx_inicio_expandido -= 1
        else:
            break
    
    idx_fim_expandido = fim_idx
    while idx_fim_expandido < len(calendario) - 1:
        data_proxima = calendario[idx_fim_expandido + 1]
        if classificacao.get(data_proxima) in ['fim_semana', 'feriado']:
            idx_fim_expandido += 1
        else:
            break
    
    periodo_completo = calendario[idx_inicio_expandido:idx_fim_expandido + 1]
    
    # Contar dias por tipo
    dias_uteis = sum(1 for d in janela_base if classificacao[d] == 'dia_util')
    fins_semana = sum(1 for d in periodo_completo if classificacao[d] == 'fim_semana')
    feriados = sum(1 for d in periodo_completo if classificacao[d] == 'feriado')
    
    resultado = {
        'janela_inicio': janela_base[0],
        'janela_fim': janela_base[-1],
        'periodo_inicio': periodo_completo[0],
        'periodo_fim': periodo_completo[-1],
        'dias_corridos': dias_corridos,
        'dias_uteis_usados': dias_uteis,
        'dias_total_folga': len(periodo_completo),
        'fins_semana': fins_semana,
        'feriados': feriados,
        'periodo_completo': periodo_completo,
        'janela_base': janela_base
    }
    
    return resultado, None

print("Função de processamento definida.")

## 6. Interface do Usuário

In [ ]:
def otimizar_ferias(dias_corridos):
    print(f"\n{'='*60}")
    print(f"Otimizando {dias_corridos} dias corridos de férias")
    print(f"{'='*60}\n")
    
    modelo, y, janela_possivel, dias_corridos_total = resolver_otimizacao_férias(
        dias_corridos, calendario, classificacao
    )
    
    resultado, erro = processar_solucao(modelo, y, janela_possivel, dias_corridos_total, calendario, classificacao)
    
    if erro:
        print(f"Erro: {erro}")
        return None
    
    print("="*60)
    print("RESULTADO")
    print("="*60)
    print(f"\n📅 Janela de férias (dias corridos): {resultado['dias_corridos']} dias")
    print(f"   De: {resultado['janela_inicio'].strftime('%d/%m/%Y')}")
    print(f"   Até: {resultado['janela_fim'].strftime('%d/%m/%Y')}")
    
    print(f"\n🎯 Período TOTAL de folga: {resultado['dias_total_folga']} dias")
    print(f"   De: {resultado['periodo_inicio'].strftime('%d/%m/%Y')}")
    print(f"   Até: {resultado['periodo_fim'].strftime('%d/%m/%Y')}")
    
    print(f"\n📊 Detalhes:")
    print(f"   Dias úteis usados: {resultado['dias_uteis_usados']}")
    print(f"   Fins de semana aproveitados: {resultado['fins_semana']}")
    print(f"   Feriados aproveitados: {resultado['feriados']}")
    
    dias_bonus = resultado['dias_total_folga'] - resultado['dias_corridos']
    print(f"\n💰 Bônus: {dias_bonus} dias extras de folga!")
    
    return resultado

# Exemplo: 10 dias corridos
resultado = otimizar_ferias(10)

## 7. Visualização Detalhada

In [ ]:
def dia_semana_pt(data):
    dias = {0: 'Seg', 1: 'Ter', 2: 'Qua', 3: 'Qui', 4: 'Sex', 5: 'Sáb', 6: 'Dom'}
    return dias[data.weekday()]

if resultado:
    print("\n" + "="*60)
    print("CALENDÁRIO DETALHADO")
    print("="*60)
    
    # Marcar onde começa e termina a janela de férias
    janela_inicio = resultado['janela_inicio']
    janela_fim = resultado['janela_fim']
    
    print(f"\nPeríodo completo ({resultado['dias_total_folga']} dias):")
    print("-" * 60)
    
    for data in resultado['periodo_completo']:
        tipo = classificacao.get(data)
        dia_pt = dia_semana_pt(data)
        
        # Identificar se está dentro da janela de férias
        if janela_inicio <= data <= janela_fim:
            marcador = "[FÉRIAS]"
        else:
            marcador = "[BÔNUS]"
        
        emoji = {'dia_util': '📅', 'fim_semana': '🏖️', 'feriado': '🎉'}[tipo]
        print(f"{emoji} {data.strftime('%d/%m/%Y')} ({dia_pt}) - {tipo:12s} {marcador}")

## 8. Testes com Diferentes Valores

In [ ]:
# Testar com diferentes quantidades
for dias in [5, 10, 15, 20]:
    print(f"\n\n{'#'*60}")
    resultado = otimizar_ferias(dias)